In [1]:
import pickle
import numpy as np
import gym
import matplotlib.pyplot as plt
from gym import TimeLimit
from itertools import product

import numpy as np
import gym
import matplotlib.pyplot as plt

from gym.envs.classic_control import MountainCarEnv
from typing import List, Optional
from contextlib import closing
from io import StringIO
from os import path
from typing import List, Optional

import numpy as np

#import gymnasium as gym
#from gymnasium import Env, spaces, utils
#from gymnasium.envs.toy_text.utils import categorical_sample
#from gymnasium.error import DependencyNotInstalled
#from gymnasium.utils import seeding

import pandas as pd

import gym
from gym import spaces
from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled





# new environment for updated reset staps

In [ ]:


class ExtendedMountainCarEnv(MountainCarEnv):
    
    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
        state: Optional[list] = None,
    ):
        super().reset(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        low, high = utils.maybe_parse_reset_bounds(options, -0.6, -0.4)
        
        #print(state)
        
        if type(state) !=None:
            
            self.state = state
        else:
            self.state = np.array([self.np_random.uniform(low=low, high=high), 0])

        if self.render_mode == "human":
            self.render()
        return np.array(self.state, dtype=np.float32), {}    
    
    
    

# descritization modules and transition matrix generator

In [978]:

#c
max_episode_steps = 200
env = ExtendedMountainCarEnv(#render_mode = 'human'
                            )


ExtendedMountainCarEnv_ref = gym.register(
    id="MountainCarEnv-v1_cust",
    entry_point=ExtendedMountainCarEnv,
)

env = gym.make('MountainCarEnv-v1_cust', render_mode=None,max_episode_steps=max_episode_steps)



env.spec.reward_threshold =-110.0




# Discretize the state space
num_states_per_dim = 5


def discretize_environment(env,dim = 5):



    low, high = env.observation_space.low, env.observation_space.high


    bin_width = (high - low) / (dim-1)
    state_space = np.zeros((dim, dim, high.shape[0] ))

    

    for i in range(dim):
        for j in range(dim):
            state_space[i, j] = low[0] + i * bin_width[0], low[1] + j * bin_width[1]
    state_space.shape
    
    
    return low,high,bin_width,state_space




def mapper_state(state, seriaized):
    return np.where(
(seriaized == state).sum(axis =1) == 2)[0][0]


# Discretize state
def discretize_state(state,low ,bin_width ,state_space ):
    state_adj = low + bin_width*((state - low)//bin_width)
    state_adj = state_space[np.all(state_space.astype('float16') ==  (state_adj).astype('float16'),axis=2)][0]
    return state_adj






def generate_P(env=env,dim = [5,],pickle_name='random_name',save = True, base_addr = r'./environments/',iter_=50,version=1,
               original = False,verbose = False,run_trial = 10, exploration_verions = 1):
    
    
    
    
    
    
    
    packages = { i:dict({}) for i in dim}
    
    
    for i in dim:
        
        low,high,bin_width,state_space = discretize_environment(env,dim = i)
        

        


        #env = gym.make('MountainCar-v0',render_mode='human')

        seriaized = state_space.reshape(-1,2)
        
        
        
        nA = env.action_space.n
        num_states_per_dim = i
        
        nS = num_states_per_dim*num_states_per_dim
        P = {s: {a: [] for a in range(nA)} for s in range(nS)}
        #P
        num_states = [num_states_per_dim,num_states_per_dim]
        # Initialize the state-action count
        # Initialize the transition matrix
        transition_matrix = np.zeros((nS, env.action_space.n, nS))

        #reward_matrix = np.zeros((nS, env.action_space.n, nS))


        terminate_matrix = np.zeros((nS))
        # Initialize the state-action count
        state_action_count = np.zeros((nS, env.action_space.n))


        packages[i] = dict(
        

                            low = low,
                            high = high,
                            bin_width = bin_width,
                            state_space = state_space,
                            seriaized = seriaized
                            ,nA = nA
                            ,nS = nS
                            ,P = P
                            ,num_states = num_states
                            ,transition_matrix = transition_matrix
                            ,terminate_matrix = terminate_matrix
                            ,state_action_count = state_action_count                
            
            




                            )        
        
        
        
        
        
        


    # Estimate the transition probabilities

    
    

    for episode in range(iter_):
        
        
        
        if exploration_verions ==1:

            if original:
                state = env.reset()[0]
            else:
                
                
                state = env.reset(state = np.array([np.random.uniform(low=low[0], high=high[0]), np.random.uniform(low=low[1], high=high[1])]))[0]
                
                
                #state = env.reset(state =packages[dim[-1]]['seriaized'][np.random.choice(len(packages[dim[-1]]['seriaized']) )] )[0]



            #print(res)

            done = False
            truncated = False
            steps = 0





            while not (done or truncated):

                steps+=1
                action = int(env.action_space.sample())
                next_state, reward, done,truncated, info = env.step(action)
                #next_state = discretize_state(next_state,low,bin_width,state_space)#discretize_state(next_state)



                #print(state[0], state[1], action, next_state[0], next_state[1])

                for i in dim:#packages.keys():

                    state_1 = discretize_state(state,
                                             packages[i]['low'],packages[i]['bin_width'],packages[i]['state_space'])#discretize_state(env.reset())

                    next_state_1 = discretize_state(next_state,
                                             packages[i]['low'],packages[i]['bin_width'],packages[i]['state_space'])

                    s1 = mapper_state(state_1,packages[i]['seriaized'])
                    s2 = mapper_state(next_state_1,packages[i]['seriaized'])
                    packages[i]['transition_matrix'][s1, action, s2] += 1
                    #reward_matrix[s1, action, s2] += reward
                    packages[i]['terminate_matrix'][s2] += done
                    packages[i]['state_action_count'][s1, action] += 1
                state = next_state

            if (verbose):
                print(steps)
            #env.render()
        
        else: # exploration_verions ==2:

            for s_1 in packages[dim[-1]]['seriaized']:#range(packages[dim[-1]]['nS']):          

                
                for action in range(packages[dim[-1]]['nA']):
                                    
                    if original:
                        state = env.reset()[0]
                    else:
                        
                        
                        try:
                            state = env.reset(state =s_1)[0]#packages[dim[-1]]['seriaized'][s_1] )[0]
                        except IOError:
                            print()
                        next_state, reward, done,truncated, info = env.step(action)
                        
                        
                        for i in packages.keys():

                            state_1 = discretize_state(state,
                                                     packages[i]['low'],packages[i]['bin_width'],packages[i]['state_space'])#discretize_state(env.reset())

                            next_state_1 = discretize_state(next_state,
                                                     packages[i]['low'],packages[i]['bin_width'],packages[i]['state_space'])

                            s1 = mapper_state(state_1,packages[i]['seriaized'])
                            s2 = mapper_state(next_state_1,packages[i]['seriaized'])
                            packages[i]['transition_matrix'][s1, action, s2] += 1
                            #reward_matrix[s1, action, s2] += reward
                            packages[i]['terminate_matrix'][s2] += done
                            packages[i]['state_action_count'][s1, action] += 1
                        state = next_state                        

                        
                    
        
        
        
        
    search_func = lambda x,y: ([[x,y-1],[x,y+1],[x-1,y],[x+1,y]])

  
        
        
        
        
    for l in dim:    

        (nS,nA,state_action_count, transition_matrix,
        
        terminate_matrix
        ) = (
        
        
        packages[l]['nS']
        ,packages[l]['nA']
        
        ,packages[l]['state_action_count']
        , packages[l]['transition_matrix']
        
        , packages[l]['terminate_matrix']
        
        
        )
        
        
        
        
        for i in range(nS):
            for j in range(nA):

                if packages[l]['state_action_count'][i,j]!=0:

                    
                    
                    
                        #if version==1:
                        #    for k in range(nS):
                        #        if packages[l]['transition_matrix'][i,j,k]!=0:
                        #            reward_matrix[i,j,
                        #                k] /= packages[l]['transition_matrix'][i,j,k]

                        packages[l]['transition_matrix'][i,j,
                                    :] /= packages[l]['state_action_count'][i,j]



        P_ = { i:
               { 

                  j: [
                   (packages[l]['transition_matrix'][i,j,k],k,-1 #reward_matrix[i,j,k]#
                    , packages[l]['terminate_matrix'][k]>0  )

                   for k in range(nS) if packages[l]['transition_matrix'][i,j,k]>0






                   ]



                   for j in range(nA)}           for i in range(nS)}


        
        
        
        
        
        not_don = True
        
        k=0
        
        gap = True
        
        
        #running complex fill
        
        if version==1:
        
            for running in range(run_trial):



                gap = False



                for i in range(nS):
                    for j in range(nA):  
                        if len(P_[i][j]) == 0:


                                for lam in search_func(i,j):
                                    if (i in [0,nS-1]) or (j in [0,nA-1]):
                                        pass#gap = True
                                    elif len(P_[lam[0]][lam[1]]) == 0:
                                        pass#gap = True

                                    else:
                                        P_[i][j] = P_[lam[0]][lam[1]]
                            
                            

        for i in range(nS):
            for j in range(nA):  
                if len(P_[i][j]) == 0:
                        P_[i][j] = [(1.0, -1, -1.0, True)]


                                


                        
                
                    
                        
                    
                    
                    
        packages[l]['P'] = P_

        

    if save:
        with open(base_addr+pickle_name+'_envP_config.pkl','wb') as fi:
            pickle.dump(
                                    packages

                                    ,fi)
    
    
    
    return packages

    
    
    
    

In [886]:
env = gym.make('MountainCar-v0', render_mode='None',max_episode_steps = 10)
env._max_episode_steps

10

# Generate Transition matrix and helpful data

In [979]:
#env = ExtendedMountainCarEnv(#render_mode = 'human'
#                            )


ExtendedMountainCarEnv_ref = gym.register(
    id="MountainCarEnv-v1_cust",
    entry_point=ExtendedMountainCarEnv,
)


max_episode_steps = 200
env = gym.make('MountainCarEnv-v1_cust', render_mode=None,max_episode_steps=max_episode_steps)

env.spec.reward_threshold =-110.0



resp_5 = generate_P(env=env,dim = [5,10,15,20,25,30]
                    ,pickle_name='MountainCar',save = True, verbose = False,
                    
                    base_addr = r'./environment/',iter_=10000, original=False,version=1,run_trial=10,exploration_verions=1)